<a href= "https://hrs.isr.umich.edu/sites/default/files/meta/2002/core/codebook/h02_00.html?_ga=2.14020593.714056361.1676427746-1610833755.1676427746">codebook</a>

In [180]:
# Importing the libraries
import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [181]:
# Importing all disposition files of every wave
all_dispositon_files = glob.glob('hrs_data/disposition/*.csv')
# Create an empty dataframe to store all the dataframes
dispositon_data_full = pd.DataFrame()
# Loop through all the files and read them into a list of dataframes
for filename in all_dispositon_files:
    df_disposition = pd.read_csv(filename, index_col=None, header=0)
    dispositon_data_full = pd.concat([dispositon_data_full, df_disposition], axis=0, ignore_index=True)
# Concatenate all the dataframes into one dataframe
dispositon_data = dispositon_data_full.iloc[:, :2].copy()
# As these are all the disposition files, we can set deceased to 1
dispositon_data['deceased'] = 1
dispositon_data

,HHID,PN,deceased
0,10475,10,1
1,10592,10,1
2,11219,10,1
3,11575,20,1
4,11626,10,1
...,...,...,...
10027,213379,10,1
10028,213410,10,1
10029,213418,10,1
10030,213428,10,1


In [190]:
# Again, import all the core files in the baseline wave 2002
all_core_files = glob.glob('hrs_data/*.csv')
# Create an empty dataframe to concatenate with all the dataframes
core_data_raw = pd.DataFrame()
# Loop through all the files and read them into a dataframe
for filename in all_core_files:
    dfcore = pd.read_csv(filename, index_col=None, header=0)
    col_to_merge = dfcore.columns.difference(core_data_raw.columns)
    if filename.endswith('R.csv'):
        core_data_raw = pd.concat([core_data_raw, dfcore[col_to_merge]], axis=1)
    elif filename.endswith('H.csv'):
        dfmerge = pd.concat([dfcore[col_to_merge], dfcore['HHID']], axis=1)
        core_data_raw = core_data_raw.merge(dfmerge,
                                            on=["HHID"], how="inner")
# Select the columns we need
core_data = core_data_raw[['HHID', 'PN',
                           'HC139', 'HC005', 'HC010', 'HC001', 'HC070', 
                           'HC117', 'HC128', 'HX067_R', 'HB014A', 'HC134', 'HQ331', 'HQ376', 'HMARITAL', 'HX060_R', 'HB031A',
                           'HB053', 'HE012', 'HE046', 'HF174', 'HG092', 'HF176',
                           'HG001'
                           ]]
# Show the first 5 rows of the dataframe
core_data.head()

,HHID,PN,HC139,HC005,HC010,HC001,HC070,HC117,HC128,HX067_R,...,HMARITAL,HX060_R,HB031A,HB053,HE012,HE046,HF174,HG092,HF176,HG001
0,3,10,170.0,5.0,5.0,3.0,5.0,5.0,1.0,1936.0,...,1.0,1.0,1.0,1,5.0,12.0,5.0,20.0,1.0,5.0
1,3,20,179.0,5.0,5.0,3.0,1.0,5.0,5.0,1938.0,...,1.0,2.0,1.0,1,5.0,12.0,5.0,100.0,1.0,5.0
2,10001,10,180.0,5.0,5.0,2.0,5.0,5.0,5.0,1939.0,...,6.0,1.0,1.0,3,NaN,NaN,5.0,9998.0,3.0,5.0
3,10003,30,120.0,1.0,5.0,1.0,5.0,5.0,5.0,1956.0,...,5.0,2.0,1.0,1,NaN,NaN,5.0,0.0,0.0,5.0
4,10004,10,230.0,1.0,5.0,2.0,1.0,5.0,1.0,1939.0,...,1.0,1.0,1.0,1,1.0,95.0,1.0,200.0,1.0,5.0


In [183]:
# Merge the core data with the disposition data
df02_d = core_data.merge(dispositon_data, on=["HHID", "PN"], how="left")
# Fill the respondents not appearing in the disposition data as not deceased
df02_d['deceased'] = df02_d['deceased'].fillna(0)
# Show the breakdown of deceased and not deceased
df02_d['deceased'].value_counts()

0.0    10973
1.0     9245
Name: deceased, dtype: int64

In [184]:
# Preprocessing the continuous/ordinal variables
con_col = ['HC139', 'HC001', 'HB014A', 'HB053', 'HE046', 'HG092', 'HF176',
           'HX067_R', 'HC134', 'HQ331', 'HQ376']
# Create a new dataframe with only continuous/ordinal variables
df02_d_ord = df02_d[con_col].copy()
# Construct the household asset variable
wealth_col = ['HC134', 'HQ331', 'HQ376']
for col in wealth_col:
    # Set values of no wealth to 0
    df02_d_ord[col] = df02_d_ord[col].replace(np.nan, 0)
df02_d_ord['wealth_amt'] = df02_d_ord['HC134'] + df02_d_ord['HQ331'] \
    + df02_d_ord['HQ376']
# Scale the wealth variable to 0-100
scaler = MinMaxScaler(feature_range=(0, 100))
df02_d_ord['wealth_amt'] = scaler.fit_transform(df02_d_ord['wealth_amt']\
    .values.reshape(-1, 1))
df02_d_ord.drop(['HC134', 'HQ331', 'HQ376'], axis=1, inplace=True)
# Construct the age variable
df02_d_ord['age'] = 2002 - df02_d_ord['HX067_R']
df02_d_ord.drop(['HX067_R'], axis=1, inplace=True)
# Set values of missing ordinal variables originally coded as negative to nan
df02_d_ord[df02_d_ord < 0] = np.nan
for i in df02_d_ord.columns:
    if max(df02_d_ord[i]) == 9999:
        df02_d_ord[i] = df02_d_ord[i].replace([9999, 9998], np.nan)
    elif max(df02_d_ord[i]) == 999:
        df02_d_ord[i] = df02_d_ord[i].replace([999, 998], np.nan)
    elif max(df02_d_ord[i]) == 99:
        df02_d_ord[i] = df02_d_ord[i].replace([99, 98, 97, 95], np.nan)
    elif max(df02_d_ord[i]) == 9:
        df02_d_ord[i] = df02_d_ord[i].replace([9, 8], np.nan)
# Check the number of missing values in original ordinal variables
df02_d_ord_nona = df02_d_ord.dropna()
print("Filled {} respondents' missing ordinal variables by mean"\
    .format(len(df02_d_ord) - len(df02_d_ord_nona)))
# Fill the missing values with the mean of the variable
df02_d_ord.fillna(df02_d_ord.mean(), inplace=True)
# Name the columns
df02_d_ord.columns = ['Weight (Pounds)', 'Self-Rated Health', 'Education Yrs',
                      'Religion Importance','Number of Grandchildren', 
                      'Hours of Volunteering', 'Number of Times Socializing',
                      'Wealth', 'Age'
]
# Get the summary statistics of the ordinal variables
df02_d_ord.describe()

Filled 9043 respondents' missing ordinal variables by mean


,Weight (Pounds),Self-Rated Health,Education Yrs,Religion Importance,Number of Grandchildren,Hours of Volunteering,Number of Times Socializing,Wealth,Age
count,20218.000000,20218.000000,20218.000000,20218.000000,20218.000000,20218.000000,20218.000000,20218.000000,20218.000000
mean,171.296493,2.868444,12.100723,1.905974,6.657524,45.703950,1.406329,2.767061,68.788793
std,38.282986,1.112213,3.363827,1.359676,5.093500,165.001137,5.934359,12.135002,9.954431
min,65.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,26.000000
25%,145.000000,2.000000,11.000000,1.000000,3.000000,0.000000,1.000000,0.000000,62.000000
50%,170.000000,3.000000,12.000000,1.000000,6.657524,10.000000,1.000000,0.000000,68.788793
75%,192.000000,4.000000,14.000000,3.000000,7.000000,45.703950,2.000000,0.000000,75.000000
max,400.000000,5.000000,17.000000,5.000000,77.000000,9000.000000,365.000000,100.000000,110.000000


In [185]:
# Preprocessing the binary/categorical variables
cat_col = ['HC005', 'HC010', 'HC070', 'HC117', 'HC128', 'HMARITAL', 'HE012',
           'HF174', 'HG001',
           'HX060_R','HB031A']
df02_d_cat = df02_d[cat_col].copy()
# Check the number of missing values in original categorical variables
print("Missing values in the columns", "\n", df02_d_cat.isnull().sum())
# Fill the missing values with the mode of the variable
df02_d_cat.fillna(df02_d_cat.mode().iloc[0], inplace=True)
for i in cat_col[:3]:
    df02_d_cat[i] = df02_d_cat[i].isin([1, 3])
for i in cat_col[3:9]:
    df02_d_cat[i] = df02_d_cat[i].isin([1])
df02_d_cat['HX060_R'] = np.where(df02_d_cat['HX060_R'] == 1, "Male", "Female")
df02_d_cat['HB031A'] = np.where(df02_d_cat['HB031A'] == 1, "White", "non-White")
df02_d_cat.columns = ['Hypertension', 'Diabetes', 'Arthritis', 'Smoking',
                      'Drunk Alcohol', 'Married', 'Children Nearby',
                      'Relatives Nearby', 'Functional Limitations',
                      'Sex', 'Race']
df02_d_cat

Missing values in the columns 
 HC005        663
HC010        663
HC070        663
HC117        786
HC128        663
HMARITAL       1
HE012       2107
HF174        669
HG001        671
HX060_R     2051
HB031A        41
dtype: int64


,Hypertension,Diabetes,Arthritis,Smoking,Drunk Alcohol,Married,Children Nearby,Relatives Nearby,Functional Limitations,Sex,Race
0,False,False,False,False,True,True,False,False,False,Male,White
1,False,False,True,False,False,True,False,False,False,Female,White
2,False,False,False,False,False,False,True,False,False,Male,White
3,True,False,False,False,False,False,True,False,False,Female,White
4,True,False,True,False,True,True,True,True,False,Male,White
...,...,...,...,...,...,...,...,...,...,...,...
20213,True,False,True,False,False,False,True,False,False,Female,White
20214,True,False,True,False,False,False,True,False,False,Female,White
20215,True,False,True,False,False,False,False,False,False,Female,non-White
20216,True,False,True,False,False,True,True,False,False,Female,White


From the earlier descriptive statistics in cleaning continuous variables, we see that the variable <code>age</code> variable includes the respondents aged below 51. This is because HRS sample collection is based on the households and include the spouse or partner of the main household respondent whose age may below 51. We will remove the respondents aged below 51 in the data cleaning process.

In [186]:
hrs_clean_full = pd.concat([df02_d['deceased'], df02_d_ord, df02_d_cat], axis=1)
hrs_clean = hrs_clean_full[hrs_clean_full['Age'] >= 51]
hrs_clean

,deceased,Weight (Pounds),Self-Rated Health,Education Yrs,Religion Importance,Number of Grandchildren,Hours of Volunteering,Number of Times Socializing,Wealth,Age,...,Diabetes,Arthritis,Smoking,Drunk Alcohol,Married,Children Nearby,Relatives Nearby,Functional Limitations,Sex,Race
0,1.0,170.000000,3.000000,12.0,1.0,12.000000,20.00000,1.000000,0.0,66.000000,...,False,False,False,True,True,False,False,False,Male,White
1,1.0,179.000000,3.000000,16.0,1.0,12.000000,100.00000,1.000000,0.0,64.000000,...,False,True,False,False,True,False,False,False,Female,White
2,0.0,180.000000,2.000000,12.0,3.0,6.657524,45.70395,3.000000,0.0,63.000000,...,False,False,False,False,False,True,False,False,Male,White
4,1.0,230.000000,2.000000,16.0,1.0,6.657524,200.00000,1.000000,0.0,63.000000,...,False,True,False,True,True,True,True,False,Male,White
5,0.0,155.000000,1.000000,12.0,1.0,6.657524,40.00000,1.000000,0.0,56.000000,...,False,False,False,True,True,True,True,False,Female,White
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20213,0.0,171.296493,2.868444,0.0,1.0,20.000000,45.70395,1.406329,0.0,68.788793,...,False,True,False,False,False,True,False,False,Female,White
20214,0.0,171.296493,2.868444,14.0,1.0,2.000000,45.70395,1.406329,0.0,68.788793,...,False,True,False,False,False,True,False,False,Female,White
20215,1.0,171.296493,2.868444,12.0,3.0,8.000000,45.70395,1.406329,0.0,68.788793,...,False,True,False,False,False,False,False,False,Female,non-White
20216,1.0,171.296493,2.868444,15.0,5.0,2.000000,45.70395,1.406329,0.0,68.788793,...,False,True,False,False,True,True,False,False,Female,White


In [187]:
hrs_clean.to_csv('hrs_clean.csv', index=False)